# Time Series Imagery Data with Google Earth Engine

Our workflow is as follows:  
1. Use Sentinel-2 imagery for the time interval (2015-2024)
2. Filter the datast to cover New York City's spatial bounds over the time range
3. Preprocess the imagery
   1. Cloud masking
   2. Extracting RGB bands, NIR for roofs
4. Export GEE imagery as GeoTIFFs or numpy arrays for SAM input

## Access time-series imagery

In [ ]:
import geemap
import ee

ee.Initialize()

# Define Brooklyn boundary
brooklyn = ee.FeatureCollection("TIGER/2018/Counties").filter(
    ee.Filter.eq('NAME', 'Kings')
)

# Load Landsat imagery (1980–2024)
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(brooklyn)

# Preprocess and visualize
image = landsat.filterDate('2020-01-01', '2020-12-31').median()
Map = geemap.Map()
Map.centerObject(brooklyn)
Map.addLayer(image, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}, "Landsat")
Map


## Segment buildings

Next is to segment the buildings from the imagery with the SAM (Segment Anything) model:
1. Preprocess imagery to extract building-relevant features
2. Run SAM to segment buildings from each time-step image
3. Save segmentation masks as raster files or NetCDF format

In [ ]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# Load the SAM model
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth").cuda()
mask_generator = SamAutomaticMaskGenerator(sam)

# Load an image of Brooklyn
import cv2
image = cv2.imread("brooklyn_2020.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Generate segmentation masks
masks = mask_generator.generate(image_rgb)

# Save masks for further analysis
import numpy as np
np.save("segmentation_masks_2020.npy", masks)

## Utilize NetCDF for managing time-series data

We will combine the segmentation masks for each time step into a NetCDF file, and use xarray to handle the multi-dimensional data.

In [ ]:
import xarray as xr
import numpy as np

# Combine masks into a NetCDF structure
years = list(range(1980, 2025))
masks = [np.load(f"segmentation_masks_{year}.npy") for year in years]

ds = xr.Dataset(
    {"building_masks": (("time", "x", "y"), masks)},
    coords={"time": years, "x": range(masks[0].shape[0]), "y": range(masks[0].shape[1])}
)

ds.to_netcdf("brooklyn_building_growth.nc")

## Analyze and visualize results

We will calculate the number of buildings or total building area per year.  
We will also create plots to visualize the trends in building growth.

In [ ]:
import matplotlib.pyplot as plt

# Calculate total building area per year
building_area = ds.building_masks.sum(dim=["x", "y"])

# Plot trends
plt.figure(figsize=(10, 6))
plt.plot(ds.time, building_area, label="Total Building Area")
plt.xlabel("Year")
plt.ylabel("Building Area (pixels)")
plt.title("Urban Development in Brooklyn (1980–2024)")
plt.legend()
plt.show()